In [2]:
!pip install robustness

  Using cached https://files.pythonhosted.org/packages/6c/5f/3d4cb8f3d02abb3860b3a76c585d2c8307f2ff969dc5cfada57cf363cd7c/robustness-1.2.1.post1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/80/59/ba66ebe892ec8302048434455fb6a9944eb0a78315184f8757094ba95435/torchvision-0.5.0-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/52/d6/79f40d230895fa1ce3b6af0d22e0ac79c65175dc069c194b79cc8e05a033/dill-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/af/0c/4f41bcd45db376e6fe5c619c01100e9b7531c55791b7244815bac6eac32c/tensorboardX-2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/53/b3/cb30dd8cc1198ae3fdb5a320ca7986d7ca76e23d16415067eafebff8685f/py3nvml-0.2.6-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f8/70/757ff7669cfe9f5726c991e856892c4fb98362b1f17abd34326937356729/cox-0.1.post3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/package

torchvision 0.5.0 has requirement torch==1.4.0, but you'll have torch 1.7.0 which is incompatible.
You are using pip version 18.1, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Here we are doing the imports that will be used. This is following the tutorial from robustness, https://robustness.readthedocs.io/en/latest/example_usage/training_lib_part_1.html. This is intended to replicate the robustness command line tool.

In [1]:
from robustness.datasets import DATASETS
from robustness.model_utils import make_and_restore_model
from robustness.train import train_model
from robustness.defaults import check_and_fill_args
from robustness.tools import constants, helpers
from robustness import defaults
from robustness import cifar_models

C:\Users\alexk\Anaconda3\lib\site-packages\robustness\train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


In [2]:
import cox
from cox import utils

In [3]:
import torch as ch
from argparse import ArgumentParser
import os

When we create this as a .py script, we use argparser to pass in params that we would want to change, but here as we have this set up in a python notebook we will have hardcoded params here.

In [4]:
# parser = ArgumentParser()
# parser = defaults.add_args_to_parser(defaults.MODEL_LOADER_ARGS, parser)
# parser = defaults.add_args_to_parser(defaults.TRAINING_ARGS, parser)
# parser = defaults.add_args_to_parser(defaults.PGD_ARGS, parser)
# # Add extra arguments here
# args = parser.parse_args()

# assert args.dataset is not None, "Must provide a dataset"
# ds_class = DATASETS[args.dataset]

# args = check_and_fill_args(args, defaults.TRAINING_ARGS, ds_class)
# if args.adv_train or args.adv_eval:
#     args = check_and_fill_args(args, defaults.PGD_ARGS, ds_class)
# args = check_and_fill_args(args, defaults.MODEL_LOADER_ARGS, ds_class)

In [5]:
train_kwargs = {
    'out_dir': "train_out",
    'adv_train': 1,
    'constraint': '2',
    'eps': 0.5,
    'attack_lr': 1.5,
    'attack_steps': 20,
    'dataset': 'cifar',
    'data': '/tmp/',
    'arch': cifar_models
}

train_args = utils.Parameters(train_kwargs)

assert train_args.dataset is not None, "Must provide a dataset"
ds_class = DATASETS[train_args.dataset]

args = check_and_fill_args(train_args, defaults.TRAINING_ARGS, ds_class)
if args.adv_train or args.adv_eval:
    args = check_and_fill_args(args, defaults.PGD_ARGS, ds_class)
args = check_and_fill_args(args, defaults.MODEL_LOADER_ARGS, ds_class)

In [6]:
# Load up the dataset
data_path = os.path.expandvars(args.data)
dataset = DATASETS[args.dataset](data_path)

# Make the data loaders
train_loader, val_loader = dataset.make_loaders(args.workers,
              args.batch_size, data_aug=bool(args.data_aug))

# Prefetches data to improve performance
train_loader = helpers.DataPrefetcher(train_loader)
val_loader = helpers.DataPrefetcher(val_loader)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [7]:
model, _ = make_and_restore_model(arch=args.arch, dataset=dataset)

In [8]:
# Create the cox store, and save the arguments in a table
store = cox.store.Store(args.out_dir, args.exp_name)
args_dict = args.as_dict() if isinstance(args, utils.Parameters) else vars(args)
schema = cox.store.schema_from_dict(args_dict)
store.add_table('metadata', schema)
store['metadata'].append_row(args_dict)
model = train_model(args, model, (train_loader, val_loader), store=store)

Logging in: C:\Users\alexk\Dropbox\CSCI-4961\CSCI4961Project\train_out\8b8aec02-c3a2-4a09-871c-baba2e463e9b


ValueError: optimizer got an empty parameter list